In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import openrouteservice
import pandas as pd
from pyproj import Transformer
import folium
import sys

sys.path.append(os.path.dirname(os.getcwd()))

from src.load import load_Hauskoordinaten

In [2]:
# Load the CSV file

df = load_Hauskoordinaten()
# Display the number of rows in the dataset
num_rows = df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

Hauskoordinaten.csv already exists.
Hauskoordinaten.csv is not a ZIP file, no extraction needed.
Number of rows in the dataset: 33933


/home/philipp/Dokumente/Master_Data_Science_Fernuni_Hagen/Masterarbeit/rettungsdient-isochrone/src/load.py:12: DtypeWarning: Columns (6,8,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,encoding='iso-8859-15', sep=';')


In [3]:
# Initialize the transformer for UTM Zone 32N to WGS84
transformer = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True)

# Function to parse and transform coordinates
def transform_coordinates(rechtswert_str, hochwert_str):
    # Replace comma with dot for decimal conversion
    rechtswert_str = str(rechtswert_str).replace(',', '.')
    hochwert_str = str(hochwert_str).replace(',', '.')

    # Extract zone number and easting value
    zone_number = int(rechtswert_str[:2])  # First two digits
    easting = float(rechtswert_str[2:])    # Remaining digits

    # Convert northing to float
    northing = float(hochwert_str)

    # Transform the coordinates
    lon, lat = transformer.transform(easting, northing)
    return pd.Series([lat, lon])

# Assuming df is your DataFrame and 'Rechtswert' and 'Hochwert' are the UTM columns
df[['lat', 'lon']] = df.apply(lambda row: transform_coordinates(row['Rechtswert'], row['Hochwert']), axis=1)

# Display the first few rows of the updated DataFrame
print(df[['Rechtswert', 'Hochwert', 'lat', 'lon']].head())

     Rechtswert     Hochwert        lat       lon
0  32396848,472  5681985,307  51.279854  7.521018
1  32396835,372    5681968,7  51.279703  7.520835
2  32396889,804  5681986,048  51.279868  7.521610
3  32396867,586  5681970,397  51.279724  7.521296
4  32396968,501  5681928,678  51.279367  7.522755


In [4]:
df.to_csv("../data/interim/Hauskoordinaten_latlon.csv")